### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

baby_df = pd.read_csv('amazon_baby.csv')
baby_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [53]:
#a)
def remove_punctuation(text):
    
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

result = remove_punctuation(baby_df["review"][4])

print(result == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock')


True


In [54]:
#b)
baby_df["review"].fillna("", inplace=True)

result = baby_df["review"][38] == ""

print(result)


True


In [55]:
#c)
baby_df = baby_df[baby_df["rating"] != 3]

# Short test
print(sum(baby_df["rating"] == 3))

0


In [56]:
#d)

baby_df["rating"] = np.where(baby_df["rating"] >= 4, 1, np.where(baby_df["rating"] <= 2, -1, baby_df["rating"]))

# Test if all ratings are now either 1 or -1
#short test:
sum(baby_df["rating"]**2 != 1)

0

## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [57]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names_out())
print(X_train_example.todense())



['adore' 'and' 'apples' 'bananas' 'dislike' 'hate' 'like' 'oranges' 'they'
 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


In [58]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2 
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer. 

In [59]:
#a)
train, test = train_test_split(baby_df, train_size=0.8, test_size=0.2, random_state=9)


In [60]:
#b)
vectorizer = CountVectorizer()

X = vectorizer.fit_transform(list(train["review"]))
y = train["rating"]
X_test = vectorizer.transform(list(test["review"]))
y_test = test["rating"]

## Exercise 3 
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [61]:
#a)
model = LogisticRegression()

model.fit(X, y)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy
print("Accuracy:", accuracy)

Accuracy: 0.9310965188450121


/home/viktoryia/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [79]:
#b)

feature_names = np.array(vectorizer.get_feature_names_out())

# Get the coefficients from the trained model
coefficients = model.coef_[0]

# Create a dictionary mapping feature names to coefficients
features_coefficients = dict(zip(feature_names, coefficients))

print("10 Most Positive Words:")
print(sorted(features_coefficients.items(), key=lambda x: x[1], reverse=True)[:10])

print("\n10 Most Negative Words:")
print(sorted(features_coefficients.items(), key=lambda x: x[1])[:10])


10 Most Positive Words:
[('excellent', 2.1747716675225086), ('awesome', 2.0528488634241624), ('glad', 1.9753174663673894), ('pleased', 1.9414936933905904), ('highly', 1.9370366617061507), ('worry', 1.718902981676613), ('amazing', 1.7131096563179138), ('perfectly', 1.71118294608913), ('exactly', 1.6439039175809096), ('perfect', 1.6282539291834797)]

10 Most Negative Words:
[('worst', -2.660591699819058), ('disappointing', -2.53162876408324), ('returning', -2.32368045821339), ('useless', -2.246441393852753), ('concept', -2.2257723832048297), ('poor', -2.2020642141623616), ('returned', -2.0757130450647043), ('poorly', -2.062658501307173), ('terrible', -2.046222325989934), ('disappointed', -1.9032122008257548)]


## Exercise 4 
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [63]:
#a)

# Predict sentiment on the test data
y_pred = model.predict(X_test)

# Print the predicted sentiment for the first few reviews
for i in range(5):
    print("Review:", test["review"].iloc[i])
    print("Actual Rating:", y_test.iloc[i])
    print("Predicted Sentiment:", y_pred[i])
    print()



Review: This is a very cute toy, but I soon realized that the battery compartment cannot be accessed and the batteries cannot be replaced. I should have researched this item better.
Actual Rating: -1
Predicted Sentiment: -1

Review: This thing is well worth the price I paid, my little guy loves it!  He is 8 mos old, and big for his age.  When we first bought it, we had the seat pulled up the highest it would go, which worked for a while.  Eventually, DS got too big, so we had to figure out a way to shorten the straps.  For a while, we did as someone else suggested and put tennis balls between the straps and the underside of the three spreaders.  That worked.  My husband didn't really like that solution though, so he took a large carabiner and wrapped the main hanging strap (between the adjuster and the spreaders) around the carabiner a bunch of times.  If we want to make the seat higher off the ground, we just wrap the strap some more.  As for the bumpers, our guy is so active in this 

In [64]:
#b)
y_proba = model.predict_proba(X_test)

# Print the predicted probabilities for the first few reviews
for i in range(5):
    print("Review:", test["review"].iloc[i])
    print("Actual Rating:", y_test.iloc[i])
    print("Predicted Probabilities:", y_proba[i])
    print()
#hint: model.predict_proba()

Review: This is a very cute toy, but I soon realized that the battery compartment cannot be accessed and the batteries cannot be replaced. I should have researched this item better.
Actual Rating: -1
Predicted Probabilities: [0.75163832 0.24836168]

Review: This thing is well worth the price I paid, my little guy loves it!  He is 8 mos old, and big for his age.  When we first bought it, we had the seat pulled up the highest it would go, which worked for a while.  Eventually, DS got too big, so we had to figure out a way to shorten the straps.  For a while, we did as someone else suggested and put tennis balls between the straps and the underside of the three spreaders.  That worked.  My husband didn't really like that solution though, so he took a large carabiner and wrapped the main hanging strap (between the adjuster and the spreaders) around the carabiner a bunch of times.  If we want to make the seat higher off the ground, we just wrap the strap some more.  As for the bumpers, our 

In [65]:
most_positive_indices = np.argsort(y_proba[:, 1])[::-1][:5]

print("Five Most Positive Reviews:")
for index in most_positive_indices:
    print("Predicted Probability (Positive):", y_proba[index, 1])
    print("Actual Rating:", y_test.iloc[index])
    print("Review:", test["review"].iloc[index])
    print()

Five Most Positive Reviews:
Predicted Probability (Positive): 1.0
Actual Rating: 1
Review: A little background on me: I have three kids (5, 3 and 5 mo. at the time of the purchase). I bought this seat for my rear-facing 3 year old.  I have done a lot of research over the last 5 years on carseats.  I'm a critically minded mechanical engineer and I'm very aware of carseat safety (aspiring to find time to be CPS Tech).  We were using a Triumph Advance prior to this seat, which has as many safety features than the original Britax Marathon line... but when this one came out, I knew it was time for an upgrade.  The other seats I was looking to get were the Graco Nautilus or the Recaro Prosport.The big safety reasons I bought it are1) The steel bars.  Not that important rear-facing, but a big deal forward facing.2) The tether.  One of the few carseats that tether rearfacing, and having two points of attachment is huge in stopping the rotation of the carseat during a side impact crash for both

In [66]:
#c) 


most_negative_indices = np.argsort(y_proba[:, 0])[::-1][:5]


print("Five Most Negative Reviews:")
for index in most_negative_indices:
    print("Predicted Probability (Negative):", y_proba[index, 0])
    print("Actual Rating:", y_test.iloc[index])
    print("Review:", test["review"].iloc[index])
    print()


Five Most Negative Reviews:
Predicted Probability (Negative): 1.0
Actual Rating: -1
Review: Please see my email to the company:Hello,I am writing to voice my family's anger over your unsafe, cheap co-sleeper.  If you recall, I had a problem with my newly purchased co-sleeper back in May, which I immediately called about and was told to send the frame back.  At that time, I asked to speak to a supervisor about the situation, and was told that I would be contacted shortly.  However, Mayra was the only one who I was able to speak with, after numerous attempts to be put in contact with the supervisor.  After a huge delay due to mistakes on your end, I finally got the co-sleeper sent back to the company, after speaking with Veronica on June 13th.At this time (June 13th), I asked to speak with the manager of the company, and Veronica told me that Sharon was not in at the time, but would be in later that day.  I obviously never heard from Sharon, or anyone else from this company, for that mat

In [51]:
#d) 
accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy
print("Accuracy:", accuracy)

Accuracy: 0.9310965188450121


## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [71]:
significant_words = ['love','great','easy','old','little','perfect','loves','well','able','car','broke','less','even','waste','disappointed','work','product','money','would','return']

In [82]:
train_dict, test_dict = train_test_split(significant_words, train_size=0.8, test_size=0.2, random_state=9)

In [83]:
#a)
vectorizer_limited = CountVectorizer(vocabulary=significant_words)

X_limited = vectorizer_limited.fit_transform(list(train["review"]))
y_limited = train["rating"]
X_test_limited = vectorizer_limited.transform(list(test["review"]))
y_test_limited = test["rating"]

TypeError: list indices must be integers or slices, not str

In [78]:


limited_model = LogisticRegression()

limited_model.fit(X_limited, y_limited)

y_pred_lim = limited_model.predict(X_test_limited)
accuracy = accuracy_score(y_test_limited, y_pred_lim)

# Print the accuracy
print("Accuracy:", accuracy)

Accuracy: 0.8670804473628977


In [80]:
feature_names_lim = np.array(vectorizer_limited.get_feature_names_out())

# Get the coefficients from the trained model
coefficients_lim = limited_model.coef_[0]

# Create a dictionary mapping feature names to coefficients
features_coefficients_lim = dict(zip(feature_names_lim, coefficients_lim))

print("10 Most Positive Words:")
print(sorted(features_coefficients_lim.items(), key=lambda x: x[1], reverse=True)[:10])

print("\n10 Most Negative Words:")
print(sorted(features_coefficients_lim.items(), key=lambda x: x[1])[:10])

10 Most Positive Words:
[('loves', 1.731097403135818), ('perfect', 1.5397133165840304), ('love', 1.356228686770304), ('easy', 1.1623365780490358), ('great', 0.9260774580423731), ('well', 0.500008141667652), ('little', 0.493507416331203), ('able', 0.22523627051436343), ('old', 0.08269355389887463), ('car', 0.06523612120562435)]

10 Most Negative Words:
[('disappointed', -2.345747699127795), ('return', -2.1201045518779327), ('waste', -1.971470284399443), ('broke', -1.7032549460595479), ('money', -0.9087495801010872), ('work', -0.6441434415169854), ('even', -0.5102634367198567), ('would', -0.34569215224131866), ('product', -0.3054884478214659), ('less', -0.16782082440073806)]


In [81]:
y_pred = model.predict(X_test)

# Print the predicted sentiment for the first few reviews
for i in range(5):
    print("Review:", test["review"].iloc[i])
    print("Actual Rating:", y_test.iloc[i])
    print("Predicted Sentiment:", y_pred[i])
    print()


Review: This is a very cute toy, but I soon realized that the battery compartment cannot be accessed and the batteries cannot be replaced. I should have researched this item better.
Actual Rating: -1
Predicted Sentiment: -1

Review: This thing is well worth the price I paid, my little guy loves it!  He is 8 mos old, and big for his age.  When we first bought it, we had the seat pulled up the highest it would go, which worked for a while.  Eventually, DS got too big, so we had to figure out a way to shorten the straps.  For a while, we did as someone else suggested and put tennis balls between the straps and the underside of the three spreaders.  That worked.  My husband didn't really like that solution though, so he took a large carabiner and wrapped the main hanging strap (between the adjuster and the spreaders) around the carabiner a bunch of times.  If we want to make the seat higher off the ground, we just wrap the strap some more.  As for the bumpers, our guy is so active in this 

In [73]:
#b)


In [74]:
#c)

#hint: %time, %timeit